In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm

In [2]:
rankings = pd.read_csv("Rankings 2018 - Cleaned.csv")
matches = pd.read_csv("FIFA2018 - Cleaned.csv")

In [3]:
matches.head()

,Team1,Team2,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks
0,Russia,Saudi Arabia,Group Phase,Luzhniki Stadium,Moscow,18:00,14,6,2018,5,0,NaN
1,Egypt,Uruguay,Group Phase,Ekaterinburg Arena,Ekaterinburg,17:00,15,6,2018,0,1,NaN
2,Morocco,IR Iran,Group Phase,Saint Petersburg Stadium,St. Petersburg,18:00,15,6,2018,0,1,NaN
3,Portugal,Spain,Group Phase,Fisht Stadium,Sochi,21:00,15,6,2018,3,3,{WinTeamName} win on aggregate after regular t...
4,France,Australia,Group Phase,Kazan Arena,Kazan,13:00,16,6,2018,2,1,NaN


In [4]:
df = pd.get_dummies(matches, columns=["Team1","Team2"])

In [5]:
df.columns

Index(['Rounds', 'Stadium', 'City', 'Time', 'Day', 'Month', 'Year', 'Score1',
       'Score2', 'Remarks', 'Team1_Argentina', 'Team1_Australia',
       'Team1_Belgium', 'Team1_Brazil', 'Team1_Colombia', 'Team1_Costa Rica',
       'Team1_Croatia', 'Team1_Denmark', 'Team1_Egypt', 'Team1_England',
       'Team1_France', 'Team1_Germany', 'Team1_IR Iran', 'Team1_Iceland',
       'Team1_Japan', 'Team1_Korea Republic', 'Team1_Mexico', 'Team1_Morocco',
       'Team1_Nigeria', 'Team1_Panama', 'Team1_Peru', 'Team1_Poland',
       'Team1_Portugal', 'Team1_Russia', 'Team1_Saudi Arabia', 'Team1_Senegal',
       'Team1_Serbia', 'Team1_Spain', 'Team1_Sweden', 'Team1_Switzerland',
       'Team1_Tunisia', 'Team1_Uruguay', 'Team2_Argentina', 'Team2_Australia',
       'Team2_Belgium', 'Team2_Brazil', 'Team2_Colombia', 'Team2_Costa Rica',
       'Team2_Croatia', 'Team2_Denmark', 'Team2_Egypt', 'Team2_England',
       'Team2_France', 'Team2_Germany', 'Team2_IR Iran', 'Team2_Iceland',
       'Team2_Japan', '

In [6]:
allTeams = set(matches.Team1.unique()).union(set(matches.Team2.unique()))

In [7]:
for t in allTeams:
    col1 = "Team1_"+t
    col2 = "Team2_"+t
    df[t] = df[col1] + df[col2]

In [8]:
df.columns

Index(['Rounds', 'Stadium', 'City', 'Time', 'Day', 'Month', 'Year', 'Score1',
       'Score2', 'Remarks',
       ...
       'Colombia', 'Spain', 'Tunisia', 'France', 'Iceland', 'Peru',
       'Australia', 'Poland', 'Croatia', 'Uruguay'],
      dtype='object', length=106)

In [9]:
df = df.drop(['Team1_Argentina', 'Team1_Australia',
       'Team1_Belgium', 'Team1_Brazil', 'Team1_Colombia', 'Team1_Costa Rica',
       'Team1_Croatia', 'Team1_Denmark', 'Team1_Egypt', 'Team1_England',
       'Team1_France', 'Team1_Germany', 'Team1_IR Iran', 'Team1_Iceland',
       'Team1_Japan', 'Team1_Korea Republic', 'Team1_Mexico', 'Team1_Morocco',
       'Team1_Nigeria', 'Team1_Panama', 'Team1_Peru', 'Team1_Poland',
       'Team1_Portugal', 'Team1_Russia', 'Team1_Saudi Arabia', 'Team1_Senegal',
       'Team1_Serbia', 'Team1_Spain', 'Team1_Sweden', 'Team1_Switzerland',
       'Team1_Tunisia', 'Team1_Uruguay', 'Team2_Argentina', 'Team2_Australia',
       'Team2_Belgium', 'Team2_Brazil', 'Team2_Colombia', 'Team2_Costa Rica',
       'Team2_Croatia', 'Team2_Denmark', 'Team2_Egypt', 'Team2_England',
       'Team2_France', 'Team2_Germany', 'Team2_IR Iran', 'Team2_Iceland',
       'Team2_Japan', 'Team2_Korea Republic', 'Team2_Mexico', 'Team2_Morocco',
       'Team2_Nigeria', 'Team2_Panama', 'Team2_Peru', 'Team2_Poland',
       'Team2_Portugal', 'Team2_Russia', 'Team2_Saudi Arabia', 'Team2_Senegal',
       'Team2_Serbia', 'Team2_Spain', 'Team2_Sweden', 'Team2_Switzerland',
       'Team2_Tunisia', 'Team2_Uruguay','Remarks'],axis=1)

In [10]:
df.columns

Index(['Rounds', 'Stadium', 'City', 'Time', 'Day', 'Month', 'Year', 'Score1',
       'Score2', 'Senegal', 'Serbia', 'Mexico', 'Portugal', 'England', 'Egypt',
       'Belgium', 'IR Iran', 'Panama', 'Sweden', 'Switzerland', 'Morocco',
       'Saudi Arabia', 'Nigeria', 'Japan', 'Germany', 'Denmark', 'Argentina',
       'Korea Republic', 'Russia', 'Costa Rica', 'Brazil', 'Colombia', 'Spain',
       'Tunisia', 'France', 'Iceland', 'Peru', 'Australia', 'Poland',
       'Croatia', 'Uruguay'],
      dtype='object')

In [11]:
rankings.head()

,Rank,Country,Score,Previous Score,Imporvement,Country Abbreviations
0,1,Germany,1558,1544,0,GER
1,2,Brazil,1431,1384,0,BRA
2,3,Belgium,1298,1346,0,BEL
3,4,Portugal,1274,1306,0,POR
4,5,Argentina,1241,1254,0,ARG


In [12]:
rank = {}
points = {}
for i in range(len(rankings)):
    if rankings.Country[i] in allTeams:
        rank[rankings.Country[i]] = rankings.Rank[i]
        points[rankings.Country[i]] = rankings.Score[i]

In [13]:
for c in ['Colombia', 'Costa Rica', 'Morocco', 'Spain',
       'Uruguay', 'Switzerland', 'Peru', 'Tunisia', 'Japan', 'Nigeria',
       'Egypt', 'Mexico', 'Brazil', 'Portugal', 'England', 'Iceland',
       'IR Iran', 'Germany', 'Saudi Arabia', 'Belgium', 'Australia', 'Serbia',
       'Russia', 'Sweden', 'France', 'Korea Republic', 'Croatia', 'Senegal',
       'Argentina', 'Panama', 'Denmark', 'Poland']:
    rank_df = len(df)*[rank[c]]
    points_df = len(df)*[points[c]]
    
    df["Rank"+c] = rank_df
    df["Points"+c] = points_df

In [14]:
for c in df.columns:
    print(c)

Rounds
Stadium
City
Time
Day
Month
Year
Score1
Score2
Senegal
Serbia
Mexico
Portugal
England
Egypt
Belgium
IR Iran
Panama
Sweden
Switzerland
Morocco
Saudi Arabia
Nigeria
Japan
Germany
Denmark
Argentina
Korea Republic
Russia
Costa Rica
Brazil
Colombia
Spain
Tunisia
France
Iceland
Peru
Australia
Poland
Croatia
Uruguay
RankColombia
PointsColombia
RankCosta Rica
PointsCosta Rica
RankMorocco
PointsMorocco
RankSpain
PointsSpain
RankUruguay
PointsUruguay
RankSwitzerland
PointsSwitzerland
RankPeru
PointsPeru
RankTunisia
PointsTunisia
RankJapan
PointsJapan
RankNigeria
PointsNigeria
RankEgypt
PointsEgypt
RankMexico
PointsMexico
RankBrazil
PointsBrazil
RankPortugal
PointsPortugal
RankEngland
PointsEngland
RankIceland
PointsIceland
RankIR Iran
PointsIR Iran
RankGermany
PointsGermany
RankSaudi Arabia
PointsSaudi Arabia
RankBelgium
PointsBelgium
RankAustralia
PointsAustralia
RankSerbia
PointsSerbia
RankRussia
PointsRussia
RankSweden
PointsSweden
RankFrance
PointsFrance
RankKorea Republic
PointsKorea

In [15]:
df.head()

,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Senegal,...,RankSenegal,PointsSenegal,RankArgentina,PointsArgentina,RankPanama,PointsPanama,RankDenmark,PointsDenmark,RankPoland,PointsPoland
0,Group Phase,Luzhniki Stadium,Moscow,18:00,14,6,2018,5,0,0,...,27,838,5,1241,55,571,12,1051,8,1183
1,Group Phase,Ekaterinburg Arena,Ekaterinburg,17:00,15,6,2018,0,1,0,...,27,838,5,1241,55,571,12,1051,8,1183
2,Group Phase,Saint Petersburg Stadium,St. Petersburg,18:00,15,6,2018,0,1,0,...,27,838,5,1241,55,571,12,1051,8,1183
3,Group Phase,Fisht Stadium,Sochi,21:00,15,6,2018,3,3,0,...,27,838,5,1241,55,571,12,1051,8,1183
4,Group Phase,Kazan Arena,Kazan,13:00,16,6,2018,2,1,0,...,27,838,5,1241,55,571,12,1051,8,1183


In [16]:
sorted(df.Time.unique())

['13:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00']

In [17]:
df.Stadium.unique()

array(['Luzhniki Stadium', 'Ekaterinburg Arena',
       'Saint Petersburg Stadium', 'Fisht Stadium', 'Kazan Arena',
       'Spartak Stadium', 'Mordovia Arena', 'Kaliningrad Stadium',
       'Samara Arena', 'Rostov Arena', 'Nizhny Novgorod Stadium',
       'Volgograd Arena'], dtype=object)

In [18]:
df.Stadium.value_counts()

Saint Petersburg Stadium    7
Luzhniki Stadium            7
Kazan Arena                 6
Samara Arena                6
Fisht Stadium               6
Nizhny Novgorod Stadium     6
Rostov Arena                5
Spartak Stadium             5
Mordovia Arena              4
Kaliningrad Stadium         4
Ekaterinburg Arena          4
Volgograd Arena             4
Name: Stadium, dtype: int64

In [19]:
dS = {}
for i in range(len(df.Stadium.unique())):
    dS[df.Stadium.unique()[i]] = i
    
dT = {}
for i in range(len(sorted(df.Time.unique()))):
    dT[sorted(df.Time.unique())[i]] = i
    
dC = {}
for i in range(len(df.City.unique())):
    dC[df.City.unique()[i]] = i

df = df.replace({"Stadium": dS, "Time": dT, "City": dC})

In [20]:
df.head()

,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Senegal,...,RankSenegal,PointsSenegal,RankArgentina,PointsArgentina,RankPanama,PointsPanama,RankDenmark,PointsDenmark,RankPoland,PointsPoland
0,Group Phase,0,0,4,14,6,2018,5,0,0,...,27,838,5,1241,55,571,12,1051,8,1183
1,Group Phase,1,1,3,15,6,2018,0,1,0,...,27,838,5,1241,55,571,12,1051,8,1183
2,Group Phase,2,2,4,15,6,2018,0,1,0,...,27,838,5,1241,55,571,12,1051,8,1183
3,Group Phase,3,3,7,15,6,2018,3,3,0,...,27,838,5,1241,55,571,12,1051,8,1183
4,Group Phase,4,4,0,16,6,2018,2,1,0,...,27,838,5,1241,55,571,12,1051,8,1183


In [21]:
df = df.drop(["Day","Month","Year"], axis=1)

In [22]:
allTeams = list(allTeams)
Team_Encoded= {}

for i in range(len(allTeams)):
    Team_Encoded[allTeams[i]] = i

In [23]:
target = []

for i in range(len(matches)):
    if matches.Score1[i] > matches.Score2[i]:
        target.append(Team_Encoded[matches.Team1[i]])
    elif matches.Score2[i] > matches.Score1[i]:
        target.append(Team_Encoded[matches.Team2[i]])
    else:
        target.append(0)
        
df["Target"] = target

In [24]:
X_train = df.loc[df['Rounds'] == "Group Phase"]
X_test = df.loc[df['Rounds'] != "Group Phase"]

In [25]:
X_train.head()

,Rounds,Stadium,City,Time,Score1,Score2,Senegal,Serbia,Mexico,Portugal,...,PointsSenegal,RankArgentina,PointsArgentina,RankPanama,PointsPanama,RankDenmark,PointsDenmark,RankPoland,PointsPoland,Target
0,Group Phase,0,0,4,5,0,0,0,0,0,...,838,5,1241,55,571,12,1051,8,1183,19
1,Group Phase,1,1,3,0,1,0,0,0,0,...,838,5,1241,55,571,12,1051,8,1183,31
2,Group Phase,2,2,4,0,1,0,0,0,0,...,838,5,1241,55,571,12,1051,8,1183,7
3,Group Phase,3,3,7,3,3,0,0,0,1,...,838,5,1241,55,571,12,1051,8,1183,0
4,Group Phase,4,4,0,2,1,0,0,0,0,...,838,5,1241,55,571,12,1051,8,1183,25


In [26]:
y_train = X_train["Target"]
y_test = X_test["Target"]

In [27]:
X_train = X_train.drop(["Score1","Score2","Rounds","Target"],axis=1)
X_test = X_test.drop(["Score1","Score2","Rounds","Target"],axis=1)

In [28]:
# clf = RandomForestClassifier()
# clf.fit(X_train, y_train)

# Pred_target = clf.predict(X_test)
# accuracy_score(y_test,Pred_target)

# probas = clf.predict_proba(X_test)

In [34]:
clf = DecisionTreeClassifier()
clf.fit (X_train, y_train)

Pred_target = clf.predict(X_test)
accuracy_score(y_test,Pred_target)

0.5